<h1>Optimización de la Experiencia de Compra en Instacart: Procesamiento de Datos</h1>

Por Víctor González

<h1>Introducción</h1>
Instacart es una aplicación de entrega de alimentos que se ha convertido en una solución indispensable para simplificar la tarea de abastecer a los usuarios con productos esenciales. Desde la comodidad de sus dispositivos móviles, los usuarios pueden seleccionar los artículos que necesitan a través de la intuitiva interfaz de la aplicación de Instacart. 

Una vez que los usuarios han creado su lista de compras virtual, entran en juego los compradores personales de Instacart. Estos expertos revisan meticulosamente cada pedido, asegurándose de que se seleccionen los productos adecuados y de la más alta calidad. Desde la frescura de las frutas y verduras hasta la elección de los productos preferidos de los clientes, los compradores personales se esfuerzan por garantizar una experiencia de compra sin igual.

Utilizando avanzados algoritmos y análisis de datos, Instacart desarrolla modelos predictivos que anticipan las necesidades y preferencias de los usuarios. Estos modelos analizan el historial de compras de los usuarios, identifican patrones de comportamiento y sugieren productos relevantes que podrían interesarles. Ya sea que se trate de predecir qué productos un usuario volverá a comprar, qué nuevos productos podrían ser de su interés o qué artículos podrían complementar perfectamente su carrito de compras durante una sesión, Instacart utiliza la potencia de los datos para ofrecer una experiencia de compra personalizada y satisfactoria.

<h1>Contexto</h1>
El propósito de este proyecto es construir un DataFrame que sirva como una estructura de datos organizada, donde el user_id y el product_id actúen como índices clave. Este DataFrame contendrá una amplia gama de características (features) que hemos calculado utilizando varias características predictoras (X). Estas características predictoras se seleccionan para describir tanto las propiedades del producto como el comportamiento del usuario en relación con uno o varios productos.

Para lograr esto, realizaremos un análisis exhaustivo de los pedidos previos en el conjunto de datos proporcionado por Instant Cart. A partir de esta información, extraeremos datos relevantes que nos permitan inferir las preferencias del usuario, las tendencias de compra y otros patrones significativos. Luego, organizaremos estos datos en el DataFrame, asegurándonos de asociar cada usuario con los productos que han comprado y las características que hemos calculado para esos productos.

Partimos del concuro en <a href='https://www.kaggle.com/competitions/instacart-market-basket-analysis'>Kaggle de Instacart Market Basket Analysis</a>. El conjunto de datos consta de los pedidos de 200,000 usuarios, con cada uno realizando entre 4 y 100 pedidos. Nuestro objetivo es identificar varias variables predictoras que describen las características de un producto y el comportamiento del usuario con respecto a uno o varios productos. Estas nuevas variables se generaron al analizar los pedidos anteriores del conjunto de datos.

<h1>Código en Python</h1>

Para trabajar en Python debemos importar las librerias que vamos a utilizar para cargar los datos y poder analizarlos.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
import matplotlib.pyplot as plt

Primero cargamos todos los pedidos (previos, de entrenamiento, de prueba) con sus metadatos (hora/día del pedido, etc.).

In [15]:
orders = pd.read_csv('orders.csv')
display(orders.head())
display(orders.shape)
display(orders.groupby(['eval_set']).size())

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0


(3421083, 7)

eval_set
prior    3214874
test       75000
train     131209
dtype: int64

Ahora cargamos el order_products_prior, que incluye todos los productos que se han comprado en cada pedido previo.

In [16]:
prior = pd.read_csv('order_products__prior.csv')
display(prior.shape)
prior.head()

(32434489, 4)

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


Luego cargamos el order_products__train, que incluye todos los productos que se han comprado en el conjunto de datos de entrenamiento.

In [17]:
train = pd.read_csv('order_products__train.csv')
display(train.shape)
train.head()

(1384617, 4)

,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1


Despues cargamos el products, que incluye el catalogo de productos.

In [20]:
prod = pd.read_csv('products.csv')
display(prod.shape)
display(prod.head())

(49688, 4)

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


Realizamos una unión izquierda en el DataFrame que contiene todos los pedidos (previos, de entrenamiento, de prueba). Los resultados finales contendrán todos los pedidos, y para aquellos que son previos, tendremos filas que mostrarán el total de productos que se han comprado.

In [21]:
order_prod = orders.merge(prior, on='order_id', how='left')
display(order_prod.shape)
order_prod.head()

(32640698, 10)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0


Vamos a crear el DataFrame user_prod donde almacenaremos nuestras características finales que se calcularán a partir del DataFrame order_prod. La primera característica que creamos es el total de veces que un usuario compró un producto.

In [26]:
user_prod = order_prod.groupby(['user_id', 'product_id'])[['order_id']].count()
user_prod.columns = ['user_prod_total_bought']
user_prod = user_prod.reset_index()
display(user_prod.shape)
user_prod.head()

(13307953, 3)

,user_id,product_id,user_prod_total_bought
0,1,196.0,10
1,1,10258.0,9
2,1,10326.0,1
3,1,12427.0,10
4,1,13032.0,3


Calculamos una variable que nos permita determinar si los usuarios reordenan frecuentemente un producto (ratio de reordenamiento) y la agregamos a nuestro DataFrame

In [27]:
item_one = user_prod[user_prod.user_prod_total_bought==1].groupby('product_id')[['user_prod_total_bought']].count()
item_one.columns = ['user_prod_customers_one_shot']
item_size = user_prod.groupby('product_id')[['user_id']].count()
item_size.columns = ['user_prod_unique_customers']

userxproduct_var= item_one.merge(item_size, how='left', left_index=True, right_on='product_id')
userxproduct_var['one_shot_ratio_product'] = userxproduct_var.user_prod_customers_one_shot / userxproduct_var.user_prod_unique_customers
userxproduct_var = userxproduct_var.reset_index()

user_prod = user_prod.merge(userxproduct_var[['product_id', 'one_shot_ratio_product']],how='left')

display(user_prod.shape)
user_prod.head()

(13307953, 4)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product
0,1,196.0,10,0.417500
1,1,10258.0,9,0.447038
2,1,10326.0,1,0.478419
3,1,12427.0,10,0.470518
4,1,13032.0,3,0.520218


Calculamos una variable que nos permita determinar las últimas 5 órdenes y la agregamos al DataFrame.

In [29]:
order_prod['order_number_back'] = order_prod.groupby('user_id')['order_number'].transform(max) - order_prod.order_number +1 
order_prod5 = order_prod[order_prod.order_number_back <= 5]
last_five = order_prod5.groupby(['user_id','product_id'])[['order_id']].count()
last_five.columns = ['times_last5']
last_five['times_last5_ratio'] = last_five.times_last5 / 5

user_prod = user_prod.merge(last_five , on=['user_id', 'product_id'], how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 6)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio
0,1,196.0,10,0.417500,4.0,0.8
1,1,10258.0,9,0.447038,4.0,0.8
2,1,10326.0,1,0.478419,NaN,NaN
3,1,12427.0,10,0.470518,4.0,0.8
4,1,13032.0,3,0.520218,2.0,0.4


Calculamos una variable que nos permita determinar con qué frecuencia un cliente compró un producto después de su primera compra y la agregamos al DataFrame.

In [30]:
times = order_prod.groupby(['user_id', 'product_id'])[['order_id']].count()
times.columns = ['Times_Bought_N']
total_orders = order_prod.groupby('user_id')[['order_number']].max()
total_orders.columns = ['total_orders']
first_order_number = order_prod.groupby(['user_id', 'product_id'])[['order_number']].min()
first_order_number.columns = ['first_order_number']
first_order_number_reset = first_order_number.reset_index()
span = pd.merge(total_orders, first_order_number_reset, on='user_id', how='right')
span['Order_Range_D'] = span.total_orders - span.first_order_number + 1
order_ratio = pd.merge(times, span, on=['user_id', 'product_id'], how='left')
order_ratio['Order_Ratio_user_id_X_product_id'] = order_ratio.Times_Bought_N / order_ratio.Order_Range_D

user_prod = user_prod.merge(order_ratio , on=['user_id', 'product_id'], how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 11)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000


<h1>Crear las características del producto</h1>

Esto implica desarrollar un conjunto de atributos o características que describan distintos aspectos de los productos disponibles en nuestro conjunto de datos. Estas características pueden abarcar una amplia gama de aspectos, desde atributos físicos como el tamaño, color o peso, hasta atributos más abstractos como la popularidad del producto, la frecuencia con la que se compra, o incluso la categoría a la que pertenece.

In [31]:
product_var = order_prod.groupby('product_id')[['reordered']].mean()
product_var.columns = ['reorder_ratio']

product_var['mean_add_to_cart_order'] = order_prod.groupby('product_id')[['add_to_cart_order']].mean()
product_var.head()

order_prod = order_prod.merge(product_var, on='product_id', how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 11)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000


In [32]:
order_prod['user_max_onb'] = order_prod.groupby('user_id').order_number.transform(np.max)
display(order_prod.shape)
order_prod.head()

(32640698, 14)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,order_number_back,reorder_ratio,mean_add_to_cart_order,user_max_onb
0,2539329,1,prior,1,2,8,NaN,196.0,1.0,0.0,11,0.776480,3.721774,11
1,2539329,1,prior,1,2,8,NaN,14084.0,2.0,0.0,11,0.810982,5.792595,11
2,2539329,1,prior,1,2,8,NaN,12427.0,3.0,0.0,11,0.740735,4.760037,11
3,2539329,1,prior,1,2,8,NaN,26088.0,4.0,0.0,11,0.539041,6.495838,11
4,2539329,1,prior,1,2,8,NaN,26405.0,5.0,0.0,11,0.441516,3.116969,11


In [34]:
from collections import defaultdict
item_cnt    = defaultdict(int)
item_chance = defaultdict(int)

pid_back = uid_back = onb_back = None

for user_id, product_id, order_number, max_onb in order_prod[['user_id', 'product_id', 'order_number', 'user_max_onb']].values:
        
    if user_id==uid_back and product_id==pid_back and (order_number-onb_back==1):
        item_cnt[product_id] +=1
    if order_number!=max_onb:
        item_chance[product_id] +=1
    
    uid_back = user_id
    pid_back = product_id
    onb_back = order_number
    
item_cnt = pd.DataFrame.from_dict(item_cnt, orient='index').reset_index()
item_cnt.columns = ['product_id', 'item_first_cnt']
item_chance = pd.DataFrame.from_dict(item_chance, orient='index').reset_index()
item_chance.columns = ['product_id', 'item_first_chance']
df = pd.merge(item_cnt, item_chance, on='product_id', how='outer').fillna(0)
df['item_first_ratio'] = df.item_first_cnt/df.item_first_chance

Calculamos dos variables: una que nos permite identificar qué productos tienen la mayor probabilidad de ser reordenados y otra que indica cuál es la posición promedio de un producto en un pedido. Estas variables se agregan al DataFrame.

In [35]:
item_N2_cnt    = defaultdict(int)
item_N2_chance = defaultdict(int)
item_N3_cnt    = defaultdict(int)
item_N3_chance = defaultdict(int)
item_N4_cnt    = defaultdict(int)
item_N4_chance = defaultdict(int)
item_N5_cnt    = defaultdict(int)
item_N5_chance = defaultdict(int)

pid_back = uid_back = onb_back = None

for product_id, user_id, order_number, max_order_number in order_prod[['product_id', 'user_id', 'order_number','user_max_onb']].values:
        
    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=2 and (max_order_number-order_number) >=2:
        item_N2_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=2:
        item_N2_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=3 and (max_order_number-order_number) >=3:
        item_N3_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=3:
        item_N3_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=4 and (max_order_number-order_number) >=4:
        item_N4_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=4:
        item_N4_chance[product_id] +=1

    if product_id==pid_back and user_id==uid_back and (order_number-onb_back)<=5 and (max_order_number-order_number) >=5:
        item_N5_cnt[product_id] +=1
    if product_id==pid_back and user_id==uid_back and (max_order_number-order_number) >=5:
        item_N5_chance[product_id] +=1

    pid_back = product_id
    uid_back = user_id
    onb_back = order_number

In [36]:
item_N2_cnt = pd.DataFrame.from_dict(item_N2_cnt, orient='index').reset_index()
item_N2_cnt.columns = ['product_id', 'item_N2_cnt']
item_N2_chance = pd.DataFrame.from_dict(item_N2_chance, orient='index').reset_index()
item_N2_chance.columns = ['product_id', 'item_N2_chance']

item_N3_cnt = pd.DataFrame.from_dict(item_N3_cnt, orient='index').reset_index()
item_N3_cnt.columns = ['product_id', 'item_N3_cnt']
item_N3_chance = pd.DataFrame.from_dict(item_N3_chance, orient='index').reset_index()
item_N3_chance.columns = ['product_id', 'item_N3_chance']

item_N4_cnt = pd.DataFrame.from_dict(item_N4_cnt, orient='index').reset_index()
item_N4_cnt.columns = ['product_id', 'item_N4_cnt']
item_N4_chance = pd.DataFrame.from_dict(item_N4_chance, orient='index').reset_index()
item_N4_chance.columns = ['product_id', 'item_N4_chance']

item_N5_cnt = pd.DataFrame.from_dict(item_N5_cnt, orient='index').reset_index()
item_N5_cnt.columns = ['product_id', 'item_N5_cnt']
item_N5_chance = pd.DataFrame.from_dict(item_N5_chance, orient='index').reset_index()
item_N5_chance.columns = ['product_id', 'item_N5_chance']


df2 = pd.merge(item_N2_cnt, item_N2_chance, on='product_id', how='outer')
df3 = pd.merge(item_N3_cnt, item_N3_chance, on='product_id', how='outer')
df4 = pd.merge(item_N4_cnt, item_N4_chance, on='product_id', how='outer')
df5 = pd.merge(item_N5_cnt, item_N5_chance, on='product_id', how='outer')

df_2_3_4_5 = pd.merge(pd.merge(df2, df3, on='product_id', how='outer'),
              pd.merge(df4, df5, on='product_id', how='outer'), 
              on='product_id', how='outer').fillna(0)

df = df.merge(df_2_3_4_5, on='product_id', how='left')

df['item_N2_ratio'] = df['item_N2_cnt']/df['item_N2_chance']
df['item_N3_ratio'] = df['item_N3_cnt']/df['item_N3_chance']
df['item_N4_ratio'] = df['item_N4_cnt']/df['item_N4_chance']
df['item_N5_ratio'] = df['item_N5_cnt']/df['item_N5_chance']

In [37]:
df.fillna(0, inplace=True)
df.reset_index(drop=True, inplace=True)

df=df[['product_id', 'item_first_ratio', 'item_N2_ratio','item_N3_ratio', 'item_N4_ratio', 'item_N5_ratio' ]]

user_prod = user_prod.merge(df, on='product_id', how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 16)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,Order_Ratio_user_id_X_product_id,item_first_ratio,item_N2_ratio,item_N3_ratio,item_N4_ratio,item_N5_ratio
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,0.909091,0.049538,1.0,1.0,1.0,1.0
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,0.900000,0.025180,1.0,1.0,1.0,1.0
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,0.142857,0.020449,1.0,1.0,1.0,1.0
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,0.909091,0.028567,1.0,1.0,1.0,1.0
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,0.300000,0.009064,1.0,1.0,1.0,1.0


<h1>Crear las características de Aisle-Department</h1>

Se crean las características de Aisle-Department para cada producto lo que implica asignar a cada uno de ellos una categorización específica dentro de la estructura jerárquica de un supermercado o tienda similar. Los pasillos (Aisles) y Departamentos (Departments) son divisiones comunes en la disposición de productos en un establecimiento, y asignar estas características a cada producto nos permite entender mejor su naturaleza y su lugar en la gama de productos disponibles.

Calculamos una variable que nos permite identificar qué pasillo tiene la mayor cantidad de productos y la agregamos al DataFrame.

In [40]:
total_products=len(prod.index)

aisle_top = prod.groupby('aisle_id')[['product_id']].count()
aisle_top.columns = ['total_products_aisle']
aisle_top.head()

prod_temp = prod.merge(aisle_top, on='aisle_id', how='left')

dept_top = prod.groupby('department_id')[['product_id']].count()
dept_top.columns = ['total_products_dept']
dept_top.head()

prod_temp = prod_temp.merge(dept_top, on='department_id', how='left')

prod_temp['total_products_aisle_ratio'] = prod_temp.total_products_aisle/total_products
prod_temp['total_products_dept_ratio'] = prod_temp.total_products_dept/total_products

user_prod = user_prod.merge(prod_temp.drop(['aisle_id', 'department_id', 'product_name'],axis=1), on='product_id', how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 24)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,...,item_N4_ratio,item_N5_ratio,total_products_aisle_x,total_products_dept_x,total_products_aisle_ratio_x,total_products_dept_ratio_x,total_products_aisle_y,total_products_dept_y,total_products_aisle_ratio_y,total_products_dept_ratio_y
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,...,1.0,1.0,463,4365,0.009318,0.087848,463,4365,0.009318,0.087848
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,...,1.0,1.0,582,6264,0.011713,0.126067,582,6264,0.011713,0.126067
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,...,1.0,1.0,382,1684,0.007688,0.033891,382,1684,0.007688,0.033891
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,...,1.0,1.0,316,6264,0.006360,0.126067,316,6264,0.006360,0.126067
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,...,1.0,1.0,454,1115,0.009137,0.022440,454,1115,0.009137,0.022440


<h1>Crear características de usuario</h1>

Esta parte implica desarrollar un conjunto de atributos que describan distintos aspectos del comportamiento y las características de los usuarios en nuestro conjunto de datos. 

Calculamos variables que nos permitan obtener el tamaño promedio, máximo y mínimo del pedido para cada cliente, y las agregamos al DataFrame.

In [41]:
order_size = order_prod.groupby(['user_id', 'order_id'])[['product_id']].count()
order_size.columns = ['size'] 
results = order_size.groupby('user_id')[['size']].mean()
results.columns = ['order_size_avg']   
results = results.reset_index()

user_prod = user_prod.merge(results, on=['user_id'], how='left')
display(user_prod.shape)
user_prod.head()

(13307953, 25)

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,...,item_N5_ratio,total_products_aisle_x,total_products_dept_x,total_products_aisle_ratio_x,total_products_dept_ratio_x,total_products_aisle_y,total_products_dept_y,total_products_aisle_ratio_y,total_products_dept_ratio_y,order_size_avg
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,...,1.0,463,4365,0.009318,0.087848,463,4365,0.009318,0.087848,5.363636
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,...,1.0,582,6264,0.011713,0.126067,582,6264,0.011713,0.126067,5.363636
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,...,1.0,382,1684,0.007688,0.033891,382,1684,0.007688,0.033891,5.363636
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,...,1.0,316,6264,0.006360,0.126067,316,6264,0.006360,0.126067,5.363636
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,...,1.0,454,1115,0.009137,0.022440,454,1115,0.009137,0.022440,5.363636


Hacemos una revisión del conjunto de datos que hemos desarrollado hasta el momento para asegurarnos de su integridad, calidad y coherencia. 

In [43]:
display(user_prod.info())
user_prod.to_csv('user_prod.csv')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 25 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_id                           int64  
 1   product_id                        float64
 2   user_prod_total_bought            int64  
 3   one_shot_ratio_product            float64
 4   times_last5                       float64
 5   times_last5_ratio                 float64
 6   Times_Bought_N                    int64  
 7   total_orders                      int64  
 8   first_order_number                int64  
 9   Order_Range_D                     int64  
 10  Order_Ratio_user_id_X_product_id  float64
 11  item_first_ratio                  float64
 12  item_N2_ratio                     float64
 13  item_N3_ratio                     float64
 14  item_N4_ratio                     float64
 15  item_N5_ratio                     float64
 16  total_products_aisle_x            

None

Mantenemos únicamente los pedidos de entrenamiento y prueba, excluyendo todos los pedidos previos, aquellos que utilizamos para crear nuestras características. Esto significa que nos enfocamos exclusivamente en los pedidos que serán utilizados para entrenar y evaluar nuestro modelo, descartando los datos que ya hemos utilizado en la etapa previa de preparación de características.

In [44]:
orders_last = orders[(orders.eval_set=='train') | (orders.eval_set=='test') ]
user_prod = user_prod.merge(orders_last, on='user_id', how='left')

display(user_prod.info())
display(user_prod.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 31 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_id                           int64  
 1   product_id                        float64
 2   user_prod_total_bought            int64  
 3   one_shot_ratio_product            float64
 4   times_last5                       float64
 5   times_last5_ratio                 float64
 6   Times_Bought_N                    int64  
 7   total_orders                      int64  
 8   first_order_number                int64  
 9   Order_Range_D                     int64  
 10  Order_Ratio_user_id_X_product_id  float64
 11  item_first_ratio                  float64
 12  item_N2_ratio                     float64
 13  item_N3_ratio                     float64
 14  item_N4_ratio                     float64
 15  item_N5_ratio                     float64
 16  total_products_aisle_x            

None

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,...,total_products_dept_y,total_products_aisle_ratio_y,total_products_dept_ratio_y,order_size_avg,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,...,4365,0.009318,0.087848,5.363636,1187899,train,11,4,8,14.0
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,...,6264,0.011713,0.126067,5.363636,1187899,train,11,4,8,14.0
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,...,1684,0.007688,0.033891,5.363636,1187899,train,11,4,8,14.0
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,...,6264,0.006360,0.126067,5.363636,1187899,train,11,4,8,14.0
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,...,1115,0.009137,0.022440,5.363636,1187899,train,11,4,8,14.0


Dividimos los datos en conjunto de entrenamiento.

In [45]:
user_prod_train = user_prod[user_prod.eval_set=='train']
user_prod_train = user_prod_train.merge(train, on=['product_id', 'order_id'], how='left' )
user_prod_train = user_prod_train.drop(['order_id','eval_set', 'add_to_cart_order'], axis=1)
user_prod_train = user_prod_train.fillna(0)

display(user_prod_train.info())
display(user_prod_train.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13307953 entries, 0 to 13307952
Data columns (total 31 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_id                           int64  
 1   product_id                        float64
 2   user_prod_total_bought            int64  
 3   one_shot_ratio_product            float64
 4   times_last5                       float64
 5   times_last5_ratio                 float64
 6   Times_Bought_N                    int64  
 7   total_orders                      int64  
 8   first_order_number                int64  
 9   Order_Range_D                     int64  
 10  Order_Ratio_user_id_X_product_id  float64
 11  item_first_ratio                  float64
 12  item_N2_ratio                     float64
 13  item_N3_ratio                     float64
 14  item_N4_ratio                     float64
 15  item_N5_ratio                     float64
 16  total_products_aisle_x            

None

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,...,total_products_dept_y,total_products_aisle_ratio_y,total_products_dept_ratio_y,order_size_avg,order_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,196.0,10,0.417500,4.0,0.8,10,11,1,11,...,4365,0.009318,0.087848,5.363636,1187899,train,11,4,8,14.0
1,1,10258.0,9,0.447038,4.0,0.8,9,11,2,10,...,6264,0.011713,0.126067,5.363636,1187899,train,11,4,8,14.0
2,1,10326.0,1,0.478419,NaN,NaN,1,11,5,7,...,1684,0.007688,0.033891,5.363636,1187899,train,11,4,8,14.0
3,1,12427.0,10,0.470518,4.0,0.8,10,11,1,11,...,6264,0.006360,0.126067,5.363636,1187899,train,11,4,8,14.0
4,1,13032.0,3,0.520218,2.0,0.4,3,11,2,10,...,1115,0.009137,0.022440,5.363636,1187899,train,11,4,8,14.0


Dividimos los datos en conjunto de pruebas.

In [48]:
user_prod_test = user_prod[user_prod.eval_set=='test']
user_prod_test = user_prod_test.drop(['order_id','eval_set'], axis=1)
user_prod_test = user_prod_test.fillna(0)

display(user_prod_test.info())
display(user_prod_test.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4833292 entries, 120 to 13307884
Data columns (total 29 columns):
 #   Column                            Dtype  
---  ------                            -----  
 0   user_id                           int64  
 1   product_id                        float64
 2   user_prod_total_bought            int64  
 3   one_shot_ratio_product            float64
 4   times_last5                       float64
 5   times_last5_ratio                 float64
 6   Times_Bought_N                    int64  
 7   total_orders                      int64  
 8   first_order_number                int64  
 9   Order_Range_D                     int64  
 10  Order_Ratio_user_id_X_product_id  float64
 11  item_first_ratio                  float64
 12  item_N2_ratio                     float64
 13  item_N3_ratio                     float64
 14  item_N4_ratio                     float64
 15  item_N5_ratio                     float64
 16  total_products_aisle_x           

None

,user_id,product_id,user_prod_total_bought,one_shot_ratio_product,times_last5,times_last5_ratio,Times_Bought_N,total_orders,first_order_number,Order_Range_D,...,total_products_dept_ratio_x,total_products_aisle_y,total_products_dept_y,total_products_aisle_ratio_y,total_products_dept_ratio_y,order_size_avg,order_number,order_dow,order_hour_of_day,days_since_prior_order
120,3,248.0,1,0.720492,0.0,0.0,1,13,2,12,...,0.126067,582,6264,0.011713,0.126067,6.769231,13,5,15,11.0
121,3,1005.0,1,0.687259,1.0,0.2,1,13,10,4,...,0.087848,894,4365,0.017992,0.087848,6.769231,13,5,15,11.0
122,3,1819.0,3,0.653128,0.0,0.0,3,13,4,10,...,0.108095,493,5371,0.009922,0.108095,6.769231,13,5,15,11.0
123,3,7503.0,1,0.616269,0.0,0.0,1,13,3,11,...,0.126067,582,6264,0.011713,0.126067,6.769231,13,5,15,11.0
124,3,8021.0,1,0.559164,0.0,0.0,1,13,2,12,...,0.062087,322,3085,0.006480,0.062087,6.769231,13,5,15,11.0


Guardamos ambos conjuntos de datos.

In [49]:
user_prod_train.to_csv('user_prod_train.csv')

In [50]:
user_prod_test.to_csv('user_prod_test.csv')